In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload
import csv_to_db as c
import warnings
import wrangle as w
from column_renamer import rename_columns,drop_cols
import rdflib
from typing import List
warnings.simplefilter("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
coaster_df = w.acquire_coaster_from_csv()
coaster_df.head().T

,0,1,2,3,4
name,Goudurix,Dream catcher,Alucinakis,Anaconda,Azteka
material_type,Steel,Steel,Steel,Wooden,Steel
seating_type,Sit Down,Suspended,Sit Down,Sit Down,Sit Down
speed,75.0,45.0,30.0,85.0,55.0
height,37.0,25.0,8.0,35.0,17.0
length,950.0,600.0,250.0,1200.0,500.0
num_inversions,7.0,0.0,0.0,0.0,0.0
manufacturer,Vekoma,Vekoma,Zamperla,William J. Cobb,Soquet
park,Parc Asterix,Bobbejaanland,Terra Mítica,Walygator Parc,Le Pal
status,status.operating,status.operating,status.operating,status.operating,status.operating


In [3]:
coaster_df[coaster_df.status == 'status.rumored']

,name,material_type,seating_type,speed,height,length,num_inversions,manufacturer,park,status
2591,Unknown,Steel,Sit Down,NaN,NaN,NaN,0.0,B&M,Kings Island,status.rumored
2592,Unknown Launch Coaster,Steel,Sit Down,90.0,NaN,NaN,2.0,Premier Rides,SeaWorld Orlando,status.rumored
2601,Unnamed Coaster,Steel,Sit Down,NaN,NaN,NaN,0.0,Intamin,Vulcania,status.rumored
2652,Unknown STC,Steel,Inverted,NaN,NaN,NaN,3.0,Vekoma,Tayto Park,status.rumored
2653,Unknown Family Shuttle,Steel,Sit Down,NaN,NaN,NaN,0.0,Vekoma,Tayto Park,status.rumored
2667,Primordial,na,na,NaN,NaN,NaN,0.0,na,Lagoon,status.rumored
2668,Unnamed wooden coaster,Wooden,Sit Down,NaN,30.0,NaN,0.0,GCI,Fantasilandia,status.rumored
2669,Unknown Blitz Coaster (Jurassic Park),Steel,Sit Down,NaN,NaN,NaN,0.0,Intamin,Universal Studios Islands of Adventure,status.rumored
2679,Unknown Wooden Coaster,na,na,NaN,NaN,NaN,0.0,GCI,SeaWorld San Antonio,status.rumored
2682,Circus Balancé,Steel,Sit Down,NaN,NaN,NaN,0.0,Intamin,Efteling,status.rumored


In [4]:
coaster_df.material_type = coaster_df.material_type.astype('category')
coaster_df.seating_type = coaster_df.seating_type.astype('category')
coaster_df.status = coaster_df.status.str[7:]
coaster_df.iloc[2591,[0,3,4,5,6,9]] = pd.Series({'name':'Orion','speed':52.0,'height':87.4776,'length':1621.841,'num_inversions':0.0,'status':'operating'})
coaster_df.iloc[2592,[0,3,4,5,6,9]] = pd.Series({'name':'Icebreaker','speed':52.0,'height':93.0,'length':1900.0,'num_inversions':0.0,'status':'operating'})
coaster_df.iloc[2679,[0,3,4,5,6,9]] = pd.Series({'name':'Texas Sting Ray','speed':55.0,'height':100.0,'length':3379.0,'num_inversions':0,'status':'operating'})
coaster_df.iloc[2669,[0,3,4,5,6,9]] = pd.Series({'name':'Velocicoaster','speed':7.0,'height':155.0,'length':4700.0,'num_inversions':4.0,'status':'operating'})
coaster_df.iloc[2353,[3,4,5,6,9]] = pd.Series({'speed':22.4,'height':19.7,'length':984.3,'num_inversions':0.0,'status':'operating'})

In [5]:
coaster_df = coaster_df.dropna(subset=['speed','height','length'])
coaster_df[~coaster_df.manufacturer.str.contains('na')]

,name,material_type,seating_type,speed,height,length,num_inversions,manufacturer,park,status
0,Goudurix,Steel,Sit Down,75.0,37.0,950.0,7.0,Vekoma,Parc Asterix,operating
1,Dream catcher,Steel,Suspended,45.0,25.0,600.0,0.0,Vekoma,Bobbejaanland,operating
2,Alucinakis,Steel,Sit Down,30.0,8.0,250.0,0.0,Zamperla,Terra Mítica,operating
3,Anaconda,Wooden,Sit Down,85.0,35.0,1200.0,0.0,William J. Cobb,Walygator Parc,operating
4,Azteka,Steel,Sit Down,55.0,17.0,500.0,0.0,Soquet,Le Pal,operating
...,...,...,...,...,...,...,...,...,...,...
2751,Wild Lightnin',Steel,Sit Down,40.0,15.0,359.0,0.0,L&T Systems,Lake Winnie,operating
2753,Cannon Ball,Wooden,Sit Down,80.0,21.0,692.0,0.0,Philadelphia Toboggan Coaster,Lake Winnie,operating
2754,Hell Cat,Wooden,Sit Down,90.0,33.0,793.0,0.0,S&S,Clementon Park,operating
2765,African Spin,Steel,Sit Down,30.0,7.0,113.0,0.0,SBF,Tatzmania Löffingen,operating


In [11]:
coaster_df.status = coaster_df.status.astype('category')
coaster_df.status.value_counts()

operating             1087
closed.definitely      170
announced               19
construction            17
retracked               16
relocated               15
closed.temporarily       8
unknown                  4
Name: status, dtype: int64

In [15]:
alternate.shape,coaster_df.shape

((1087, 56), (1336, 10))